In [2]:
import numpy as np
import pandas_ta as ta
import pandas as pd
from datetime import datetime, timedelta, timezone
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover

In [3]:
def data_backtest(data):
    new_data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    new_data.index = pd.to_datetime(data['Date'])
    new_data.index.name = None
    return new_data

def split_data(data):
    """
    Hàm này chia dữ liệu thành 2 phần: tập huấn luyện và tập hold out.

    Args:
    data (pandas.DataFrame): DataFrame chứa dữ liệu cần chia.

    Returns:
    pandas.DataFrame: DataFrame chứa dữ liệu tập huấn luyện.
    pandas.DataFrame: DataFrame chứa dữ liệu tập giữ lại.
    """
    # Chia dữ liệu thành 3 phần
    new_part = np.array_split(data, 3)

    # Access each part individually
    hold_out = new_part[2]
    train_data = pd.concat([new_part[0], new_part[1]], axis=0)

    return train_data, hold_out

def process_data(df):
    data = df.copy()
    data = data[~data.index.duplicated(keep='first')] # Handling duplicate
    
    data['Date'] = [str(i)[:10] for i in data.index]
    data['time'] = [str(i)[11:] for i in data.index]
    data_model = data.pivot(index = 'Date', columns = 'time', values = ['Open','High','Low','Close','Volume']).ffill(axis = 1).stack().reset_index() # Handling missing values

    return data_model

In [4]:
df_min = pd.read_csv('vn30f1m_3min.csv')
df_hour = pd.read_csv('vn30f1m_1hour.csv')
df_day = pd.read_csv('vn30f1m_1day.csv')

data_min = data_backtest(df_min)
data_hour = data_backtest(df_hour)
data_day = data_backtest(df_day)

train_min, valid_min = split_data(data_min)
train_hour, valid_hour = split_data(data_hour)
train_day, valid_day = split_data(data_day)

# 1-day strategies

## Breakout with Bollinger Bands

In [94]:
import talib
import numpy as np
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# Bollinger Breakout Strategy
class BollingerBreakout(Strategy):
    bb_period = 20  # Sử dụng bb_period tốt nhất
    risk_reward_ratio = 2  # Tỷ lệ R:R = 1:2
    min_diff = 0.001

    def init(self):
        # Sử dụng talib để tính Bollinger Bands
        self.upper_band, self.middle_band, self.lower_band = self.I(
            talib.BBANDS, self.data.Close, timeperiod=self.bb_period, nbdevup=1.5, nbdevdn=1.5, matype=0)

    def next(self):
        price_diff = self.data.Close[-1] - self.lower_band[-1]
        
        # Kiểm tra sự khác biệt đủ lớn để thực hiện giao dịch
        if price_diff > self.min_diff:
            sl = self.data.Close[-1] - price_diff
            tp = self.data.Close[-1] + price_diff * self.risk_reward_ratio
            
            # Điều kiện Stop Loss phải nhỏ hơn Take Profit và sự khác biệt đủ lớn
            if sl < tp and (tp - sl) > self.min_diff:
                if crossover(self.data.Close, self.upper_band):  # Điều kiện giá vượt dải trên
                    if sl < self.data.Close[-1] < tp:
                        self.buy(size=1/6, sl=sl, tp=tp)
                elif crossover(self.lower_band, self.data.Close):  # Điều kiện giá vượt dải dưới
                    self.sell(size=1/6, sl=tp, tp=sl)


# Hàm backtest với hàm optimize
def run_backtest_optimize(strategy_class, train_data, val_data, bb_period_range):
    # Backtest và tối ưu hóa trên tập train
    print("Optimizing on training set...")
    bt_train = Backtest(train_data, strategy_class, cash=3_000, commission=.01, margin=0.13, hedging=True)
    stats_train = bt_train.optimize(
        bb_period=bb_period_range,  # Phạm vi tối ưu bb_period
        maximize='# Trades',  # Tối ưu lợi nhuận Return [%]
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation
    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, strategy_class, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()
    
    return stats_train, stats_val

# Split data (giả sử bạn đã có train_day và valid_day từ trước)
# Example: Chạy trên dữ liệu train_day và valid_day
bb_period_range = range(10, 100, 5)  # Điều chỉnh khoảng giá trị bb_period
stats_train, stats_val = run_backtest_optimize(BollingerBreakout, train_day, valid_day, bb_period_range)

Optimizing on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/9 [00:00<?, ?it/s]

Backtesting on validation set...


In [95]:
stats_train

Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   76.071785
Equity Final [$]                  4991.377544
Equity Peak [$]                   7830.641449
Return [%]                          66.379251
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                   13.791483
Volatility (Ann.) [%]               57.016863
Sharpe Ratio                         0.241884
Sortino Ratio                        0.398773
Calmar Ratio                         0.215697
Max. Drawdown [%]                  -63.939278
Avg. Drawdown [%]                  -10.537821
Max. Drawdown Duration      841 days 00:00:00
Avg. Drawdown Duration       74 days 00:00:00
# Trades                                   90
Win Rate [%]                             50.0
Best Trade [%]                       26.75012
Worst Trade [%]                     -9.417152
Avg. Trade [%]                    

In [96]:
stats_val 

Start                     2022-08-31 09:00:00
End                       2024-08-30 09:00:00
Duration                    730 days 00:00:00
Exposure Time [%]                    87.42515
Equity Final [$]                  4478.860227
Equity Peak [$]                   4937.551338
Return [%]                          49.295341
Buy & Hold Return [%]                2.998686
Return (Ann.) [%]                   22.432045
Volatility (Ann.) [%]               57.192678
Sharpe Ratio                         0.392219
Sortino Ratio                        0.691085
Calmar Ratio                         0.582675
Max. Drawdown [%]                  -38.498409
Avg. Drawdown [%]                     -8.5643
Max. Drawdown Duration      359 days 00:00:00
Avg. Drawdown Duration       47 days 00:00:00
# Trades                                   22
Win Rate [%]                        72.727273
Best Trade [%]                      19.831275
Worst Trade [%]                     -9.065107
Avg. Trade [%]                    

## Breakout with Volume

In [107]:
import talib
import numpy as np
import pandas as pd
from backtesting import Backtest, Strategy

# Chiến thuật Break-out với Volume
class VolumeBreakout(Strategy):
    risk_reward_ratio = 2  # Tỷ lệ R:R = 1:2
    volume_period = 20  # Số phiên trung bình khối lượng
    min_diff = 0.001  # Sự khác biệt tối thiểu để đảm bảo tính hợp lý của SL và TP

    def init(self):
        # Chuyển đổi khối lượng thành kiểu float64 để phù hợp với talib
        volume_float = self.data.Volume.astype(np.float64)
        # Tính khối lượng trung bình
        self.volume_avg = self.I(talib.SMA, volume_float, timeperiod=self.volume_period)

    def next(self):
        price_diff = self.data.Close[-1] - self.data.Low[-1]

        # Kiểm tra khối lượng hiện tại có vượt khối lượng trung bình hay không và đảm bảo price_diff đủ lớn
        if self.data.Volume[-1] > self.volume_avg[-1] and price_diff > self.min_diff:
            sl = self.data.Close[-1] - price_diff
            tp = self.data.Close[-1] + price_diff * self.risk_reward_ratio

            # Kiểm tra tính hợp lý của SL và TP trước khi đặt lệnh mua (long)
            if sl < self.data.Close[-1] < tp:
                self.buy(size=1/6, sl=sl, tp=tp)

        # Đóng vị thế khi giá giảm xuống dưới mức hỗ trợ (giá thấp nhất)
        elif self.data.Close[-1] < self.data.Low[-1]:
            self.sell(size=1/6)

# Hàm để thực hiện tối ưu hóa và backtest
def run_backtest_optimize(strategy_class, train_data, val_data, volume_period_range):
    # Backtest và tối ưu hóa trên tập train
    print("Optimizing on training set...")
    bt_train = Backtest(train_data, strategy_class, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_train = bt_train.optimize(
        volume_period=volume_period_range,  # Phạm vi tối ưu volume_period
        maximize='Sharpe Ratio',  # Tối ưu Sharpe Ratio
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation
    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, strategy_class, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage
volume_period_range = range(10, 101, 5)  # Điều chỉnh khoảng giá trị volume_period
stats_train, stats_val = run_backtest_optimize(VolumeBreakout, train_day, valid_day, volume_period_range)


Optimizing on training set...


ValueError: Long orders require: SL (948.6) < LIMIT (962.53) < TP (961.8)